In [1]:
import pandas as pd
import numpy as np
import nltk
import nltk
from nltk.corpus import stopwords, wordnet
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.tokenize import word_tokenize
import re
from sklearn.model_selection import train_test_split
from gensim.models import Word2Vec
from keras.preprocessing.sequence import pad_sequences
from sklearn.ensemble import GradientBoostingClassifier, AdaBoostClassifier, VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
# from tpot import TPOTClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.metrics import classification_report, mean_squared_error
from keras.models import Sequential
from keras.layers import Embedding, SimpleRNN, GRU, LSTM, Dense, Dropout
from keras.initializers import Constant
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor, GradientBoostingRegressor
from sklearn.tree import DecisionTreeRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
import pickle as pkl
from bayes_opt import BayesianOptimization
from sklearn.model_selection import cross_val_score
import os

2024-04-18 11:08:37.215675: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-04-18 11:08:37.588742: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-18 11:08:37.588783: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-18 11:08:37.645639: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-04-18 11:08:37.769813: I tensorflow/core/platform/cpu_feature_guar

In [2]:
# df=pd.read_csv("FullyLabelled/sentiment-emotion-labelled_Dell_tweets.csv", index_col="Index")
# df.columns=["DateTime", "TweetID", "Text", "Username", "Sentiment", "SScore", "Emotion","EScore"]
# df.head()

In [3]:
# df.drop(["DateTime", "TweetID", "Username"], axis=1, inplace=True)
# df.head()

In [4]:
# df.Sentiment.unique(), df.Emotion.unique()

In [5]:
# df.Sentiment.value_counts(), df.Emotion.value_counts()

In [6]:
sdict={'neutral':1, 'positive':0, 'negative':2}
edict={'disgust':0,'anger':1, 'sadness':2, 'fear':3,'surprise':4, 'optimism':5, 'joy':6,'anticipation':7}
# df.Sentiment=df.Sentiment.replace(sdict)
# df.Emotion=df.Emotion.replace(edict)
# df.head()

In [7]:
def process_text(text):
    text = re.sub(r'http\S+|www.\S+', '', text)
    return text

# df.Text=df.Text.apply(process_text)

In [8]:
# df.Text.head(20)

In [9]:
def process_text2(text):
    stop_words = set(stopwords.words('english'))
    stemmer = PorterStemmer()
    lemmatizer = WordNetLemmatizer()

    processed_text = []

    words = word_tokenize(text)

    processed_words = []
    for word in words:
        if word not in stop_words:
            stemmed_word = stemmer.stem(word)
            lemmatized_word = lemmatizer.lemmatize(stemmed_word)
            processed_words.append(lemmatized_word)

    processed_text.append(' '.join(processed_words))

    return processed_text


In [10]:
# df.Text=df.Text.apply(process_text2)
# df.Text.head(20)

In [11]:
# tweets = df['Text'].tolist()
# tokenized_tweets = [tweet[0].split() for tweet in tweets]

# modelw2v = Word2Vec(tokenized_tweets, vector_size=300, window=5, min_count=10, workers=8)
# modelw2v.train(tokenized_tweets, total_examples=len(tokenized_tweets), epochs=20)

In [12]:
# modelw2v.wv.most_similar("ai")

In [13]:

# X = df.Text
# y = df.drop(['Text'], axis=1)

# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42, stratify=df["Emotion"])


In [14]:
# y_test.Emotion.value_counts(), y_train.Emotion.value_counts()

In [15]:
def sentence_to_vec(s, model):
    # return np.array([model.wv[word] for word in s[0].split() if word in modelw2v.wv.key_to_index])
    vecs = [model.wv[word] for word in s[0].split() if word in model.wv.key_to_index]
    return np.mean(vecs, axis=0) if vecs else np.zeros(model.vector_size)


# X_train_vec = [sentence_to_vec(s, modelw2v) for s in X_train]
# X_test_vec = [sentence_to_vec(s, modelw2v) for s in X_test]

# max_len = max(max(len(x) for x in X_train_vec), max(len(x) for x in X_test_vec))

# X_train_pad = pad_sequences(X_train_vec, maxlen=max_len, dtype='float32', padding='post')
# X_test_pad = pad_sequences(X_test_vec, maxlen=max_len, dtype='float32', padding='post')


In [16]:
# X_test_pad.shape, y_test.shape

In [17]:
# models = {
#     'Voting Classifier': VotingClassifier(estimators=[
#         ('xg', XGBClassifier(use_label_encoder=False, eval_metric='logloss', verbosity=0)),
#         ('lgbm', LGBMClassifier(verbose=0, random_state=42)),
#         ('hgb', HistGradientBoostingClassifier(random_state=42))
#     ], voting='hard'),
#     'Logistic Regression': LogisticRegression(verbose=0, max_iter=1000),
#     'Decision Tree': DecisionTreeClassifier(random_state=42),
#     'Random Forest': RandomForestClassifier(verbose=0, random_state=42),
#     'AdaBoost': AdaBoostClassifier(random_state=42),
#     'XGBoost': XGBClassifier(use_label_encoder=False, eval_metric='logloss', verbosity=0),
#     'CatBoost': CatBoostClassifier(verbose=0, random_seed=42),
#     'LightGBM': LGBMClassifier(verbose=0, random_state=42),
#     'HistGradientBoosting': HistGradientBoostingClassifier(random_state=42)
#     # 'TPOT': TPOTClassifier(generations=5, population_size=50, verbosity=0, random_state=42)
# }

# y_trains = np.array(y_train.Sentiment).reshape(-1,)

# for model_name, model in models.items():
#     print(f"Training {model_name}...")
#     model.fit(X_train_pad, y_trains)
    
#     print(f"Evaluating {model_name}...")
#     y_pred = model.predict(X_test_pad)
#     print(classification_report(y_test.Sentiment, y_pred))


In [18]:
# vocab_size = len(modelw2v.wv.key_to_index) + 1
# vector_size = modelw2v.wv.vector_size

# embedding_matrix = np.zeros((vocab_size, vector_size))
# for word, i in modelw2v.wv.key_to_index.items():
#     embedding_matrix[i] = modelw2v.wv[word]

In [19]:
# def create_model(model_type):
#     model = Sequential()
#     model.add(Embedding(vocab_size, vector_size, embeddings_initializer=Constant(embedding_matrix), trainable=True))
#     model.add(Dropout(0.5))
#     if model_type == 'RNN':
#         model.add(SimpleRNN(50))
#     elif model_type == 'GRU':
#         model.add(GRU(50))
#     elif model_type == 'LSTM':
#         model.add(LSTM(50))
    
#     model.add(Dense(1, activation='sigmoid'))
    
#     return model

In [20]:
# for model_type in ['RNN', 'GRU', 'LSTM']:
#     print(f"Training {model_type} model...")
#     model = create_model(model_type)
#     model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
#     model.fit(X_train_pad, y_trains, epochs=10, verbose=1)


In [21]:
# modele = {
#     'Logistic Regression': LogisticRegression(verbose=0, max_iter=1000),
#     'Decision Tree': DecisionTreeClassifier(random_state=42),
#     'Random Forest': RandomForestClassifier(verbose=0, random_state=42),
#     'AdaBoost': AdaBoostClassifier(random_state=42),
#     'Voting Classifier': VotingClassifier(estimators=[
#         ('xg', XGBClassifier(use_label_encoder=False, eval_metric='logloss', verbosity=0)),
#         ('lgbm', LGBMClassifier(verbose=0, random_state=42)),
#         ('hgb', HistGradientBoostingClassifier(random_state=42))
#     ], voting='hard'),
#     'XGBoost': XGBClassifier(use_label_encoder=False, eval_metric='logloss', verbosity=0),
#     'CatBoost': CatBoostClassifier(verbose=0, random_seed=42),
#     'LightGBM': LGBMClassifier(verbose=0, random_state=42),
#     'HistGradientBoosting': HistGradientBoostingClassifier(random_state=42)
#     # 'TPOT': TPOTClassifier(generations=5, population_size=50, verbosity=0, random_state=42)
# }

# y_trains = np.array(y_train.Emotion).reshape(-1,)

# for model_name, model in modele.items():
#     print(f"Training {model_name}...")
#     model.fit(X_train_pad, y_trains)
    
#     print(f"Evaluating {model_name}...")
#     y_pred = model.predict(X_test_pad)
#     print(classification_report(y_test.Emotion, y_pred, zero_division=0))

In [22]:
# modele = {
#     'Linear Regression': LinearRegression(),
#     'Decision Tree Regressor': DecisionTreeRegressor(random_state=42),
#     'Random Forest Regressor': RandomForestRegressor(random_state=42),
#     'AdaBoost Regressor': AdaBoostRegressor(random_state=42),
#     'Gradient Boosting Regressor': GradientBoostingRegressor(random_state=42),
#     'XGBoost Regressor': XGBRegressor(use_label_encoder=False, eval_metric='logloss', verbosity=0),
#     'CatBoost Regressor': CatBoostRegressor(verbose=0, random_seed=42),
#     'LightGBM Regressor': LGBMRegressor(random_state=42)
# }

# y_trains = np.array(y_train.SScore).reshape(-1,)

# for model_name, model in modele.items():
#     print(f"Training {model_name}...")
#     model.fit(X_train_pad, y_trains)
    
#     print(f"Evaluating {model_name}...")
#     y_pred = model.predict(X_test_pad)
#     print('Mean Squared Error:', mean_squared_error(y_test.SScore, y_pred))


In [23]:
# modele = {
#     'Linear Regression': LinearRegression(),
#     'Decision Tree Regressor': DecisionTreeRegressor(random_state=42),
#     'Random Forest Regressor': RandomForestRegressor(random_state=42),
#     'AdaBoost Regressor': AdaBoostRegressor(random_state=42),
#     'Gradient Boosting Regressor': GradientBoostingRegressor(random_state=42),
#     'XGBoost Regressor': XGBRegressor(use_label_encoder=False, eval_metric='logloss', verbosity=0),
#     'CatBoost Regressor': CatBoostRegressor(verbose=0, random_seed=42),
#     'LightGBM Regressor': LGBMRegressor(random_state=42)
# }

# y_trains = np.array(y_train.EScore).reshape(-1,)

# for model_name, model in modele.items():
#     print(f"Training {model_name}...")
#     model.fit(X_train_pad, y_trains)
    
#     print(f"Evaluating {model_name}...")
#     y_pred = model.predict(X_test_pad)
#     print('Mean Squared Error:', mean_squared_error(y_test.EScore, y_pred))


In [24]:
# os.system(f'cp {"FullyLabelled/sentiment-emotion-labelled_Dell_tweets.csv"} {"TempSmall.csv"}')
# os.system(f'cp {"SemiLabelled/training.1600000.processed.noemoticon.csv"} {"TempLarge.csv"}')

In [25]:
# small=pd.read_csv("TempSmall.csv", index_col="Index")
# small.columns=["DateTime", "TweetID", "Text", "Username", "Sentiment", "SScore", "Emotion","EScore"]
# small.drop(["DateTime", "TweetID", "Username"], axis=1, inplace=True)
# small.Sentiment=small.Sentiment.replace(sdict)
# small.Emotion=small.Emotion.replace(edict)
# small.Text=small.Text.apply(process_text)
# small.Text=small.Text.apply(process_text2)

# large=pd.read_csv("TempLarge.csv", names=["Sentiment", "ID2", "DT", "Q", "UserName", "Text"], encoding="ISO-8859-1")
# large.drop(["DT", "ID2", "UserName", "Q"], axis=1, inplace=True)
# large.Sentiment=large.Sentiment.replace({0:2, 4:0,2:1})
# large.Text=large.Text.apply(process_text)
# large.Text=large.Text.apply(process_text2)

# small.to_csv("TempSmall.csv")
# large.to_csv("TempLarge.csv")

# tweets = small['Text'].tolist()+large['Text'].tolist()
# tokenized_tweets = [tweet[0].split() for tweet in tweets]

# modelw2v = Word2Vec(tokenized_tweets, vector_size=300, window=5, min_count=10, workers=8)
# modelw2v.train(tokenized_tweets, total_examples=len(tokenized_tweets), epochs=20)

# pkl.dump(modelw2v, open("W2V_Model.pkl", "wb"))


In [26]:
# pkl.dump(small, open("small.pkl", "wb"))
# pkl.dump(large, open("large.pkl", "wb"))


In [36]:
small=pkl.load(open("small.pkl", "rb"))
# large=pkl.load(open("large.pkl", "rb"))
large1=pkl.load(open("large1.pkl", "rb"))
modelw2v=pkl.load(open("W2V_Model.pkl", "rb"))

In [35]:
# pkl.dump(large[:large.shape[0]//2], open("large1.pkl", "wb"))

In [34]:
# pkl.dump(large[large.shape[0]//2:], open("large2.pkl", "wb"))

In [28]:
# small=pd.read_csv("TempSmall.csv", index_col="Index")
# large=pd.read_csv("TempLarge.csv",index_col="Index", encoding="ISO-8859-1",low_memory=False)

In [37]:

X_train_vec = [sentence_to_vec(s, modelw2v) for s in small.Text]
X_test_vec = [sentence_to_vec(s, modelw2v) for s in large1.Text]

max_len = max(max(len(x) for x in X_train_vec), max(len(x) for x in X_test_vec))


In [38]:

X_train_pad = pad_sequences(X_train_vec, maxlen=max_len, dtype='float32', padding='post')
X_test_pad = pad_sequences(X_test_vec, maxlen=max_len, dtype='float32', padding='post')

y_train=small.drop(["Text"], axis=1)
y_test=large1.drop(["Text"], axis=1)


In [ ]:
X_train_pad.shape

array([[-0.31407335, -0.07926951,  0.08237156, ...,  0.04243321,
         0.07356723,  0.00521673],
       [ 0.3245958 , -0.36294833,  0.05073858, ..., -0.0060014 ,
         0.6755086 , -0.38814867],
       [-0.0997056 , -0.05142836,  0.10781164, ...,  0.29043314,
         0.41714585, -0.25970918],
       ...,
       [ 0.12978235,  0.6730353 ,  0.3436516 , ..., -0.0110016 ,
         0.4086631 , -0.6039487 ],
       [-0.09105451,  0.1899454 , -0.09428181, ...,  0.24367954,
         0.40905598, -0.01783706],
       [ 0.33882108, -0.5553169 ,  0.19569983, ...,  0.02169369,
         0.5468526 , -0.3540177 ]], dtype=float32)

In [41]:
# with open('X_train_pad.pkl', 'wb') as f:
#     pickle.dump(X_train_pad, f)

with open('X_test_pad.pkl', 'wb') as f:
    pkl.dump(X_test_pad, f)

# # Pickle the y values
# with open('y_train.pkl', 'wb') as f:
#     pickle.dump(y_train, f)

# with open('y_test.pkl', 'wb') as f:
#     pickle.dump(y_test, f)


: 

In [32]:
with open('X_train_pad.pkl', 'rb') as f:
    X_train_pad = pkl.load(f)

with open('X_test_pad.pkl', 'rb') as f:
    X_test_pad = pkl.load(f)

EOFError: Ran out of input

In [ ]:
X_test_pad

array([[ 0.05364796, -0.38689256,  0.09192634, ..., -0.309362  ,
         0.35936925, -0.41113326],
       [-0.45185354,  0.27520022,  0.3185646 , ..., -0.02325147,
         0.40133536, -0.39236057],
       [-0.03389205, -0.5166912 ,  0.120496  , ..., -0.2786832 ,
         0.09066193, -0.03940602],
       ...,
       [-0.3303782 , -0.1466112 ,  0.47284228, ...,  0.60390085,
         0.08154381, -0.28264546],
       [ 0.39953598,  0.26543802,  0.1520131 , ..., -0.40175033,
         0.84357166,  0.04084152],
       [ 0.31709138,  0.32290527, -0.21794027, ...,  0.82818097,
         0.7902313 ,  0.0528754 ]], dtype=float32)

In [ ]:
X_train_pad

array([[-0.31407335, -0.07926951,  0.08237156, ...,  0.04243321,
         0.07356723,  0.00521673],
       [ 0.3245958 , -0.36294833,  0.05073858, ..., -0.0060014 ,
         0.6755086 , -0.38814867],
       [-0.0997056 , -0.05142836,  0.10781164, ...,  0.29043314,
         0.41714585, -0.25970918],
       ...,
       [ 0.12978235,  0.6730353 ,  0.3436516 , ..., -0.0110016 ,
         0.4086631 , -0.6039487 ],
       [-0.09105451,  0.1899454 , -0.09428181, ...,  0.24367954,
         0.40905598, -0.01783706],
       [ 0.33882108, -0.5553169 ,  0.19569983, ...,  0.02169369,
         0.5468526 , -0.3540177 ]], dtype=float32)

In [ ]:
def catboost_optimization(cv_splits, X, y, mode=1):
    def function(learning_rate, depth, l2_leaf_reg, border_count):
        if mode:
            return cross_val_score(
                CatBoostClassifier(
                    learning_rate=learning_rate,
                    depth=int(round(depth)),
                    l2_leaf_reg=l2_leaf_reg,
                    border_count=int(round(border_count)),
                    loss_function="Logloss",
                    verbose=False,
                ),
                X=X,
                y=y,
                cv=cv_splits,
                scoring="accuracy",
                n_jobs=-1,
            ).mean()
        else:
            return cross_val_score(
                CatBoostRegressor(
                    learning_rate=learning_rate,
                    depth=int(round(depth)),
                    l2_leaf_reg=l2_leaf_reg,
                    border_count=int(round(border_count)),
                    loss_function="Logloss",
                    verbose=False,
                ),
                X=X,
                y=y,
                cv=cv_splits,
                scoring="accuracy",
                n_jobs=-1,
            ).mean()

    return function




In [ ]:
while len(X_test_pad) > 0:
    models = {
        "Sentiment": CatBoostClassifier(verbose=False),
        "Emotion": CatBoostClassifier(verbose=False),
        "SScore": CatBoostRegressor(verbose=False),
        "EScore": CatBoostRegressor(verbose=False),
    }

    for label in models.keys():
        if label=="SScore" or label=="EScore":
            mode=0
        else:
            mode=1
        optimizer = BayesianOptimization(
            f=catboost_optimization(5, X_train_pad, y_train[label], mode),
            pbounds={
                "learning_rate": (0.01, 1.0),
                "depth": (4, 10),
                "l2_leaf_reg": (1, 10),
                "border_count": (5, 255)
            },
            random_state=1234,
            verbose=2
        )

        optimizer.maximize(n_iter=25, init_points=2)
        params=optimizer.max["params"]
        if mode:
            m=CatBoostClassifier(**params)
        else:
            m=CatBoostRegressor(**params)
        
        m.fit(X_train_pad, y_train[label])
        
        models[label]=m

    
    predictions = {}
    for label in models.keys():
        predictions[label] = models[label].predict_proba(X_test_pad)

    indices = np.where(
        (predictions["Sentiment"][:, 1] > 0.9)
        & (predictions["Emotion"][:, 1] > 0.9)
        & (predictions["SScore"][:, 1] > 0.9)
        & (predictions["EScore"][:, 1] > 0.9)
    )

    X_train_pad = np.concatenate((X_train_pad, X_test_pad[indices]))
    y_train["Sentiment"] = np.concatenate(
        (y_train["Sentiment"], predictions["Sentiment"][indices, 1])
    )
    y_train["Emotion"] = np.concatenate(
        (y_train["Emotion"], predictions["Emotion"][indices, 1])
    )
    y_train["SScore"] = np.concatenate(
        (y_train["SScore"], predictions["SScore"][indices, 1])
    )
    y_train["EScore"] = np.concatenate(
        (y_train["EScore"], predictions["EScore"][indices, 1])
    )

    X_test_pad = np.delete(X_test_pad, indices, axis=0)
    y_test=np.delete(y_test, indices, axis=0)
    

|   iter    |  target   | border... |   depth   | l2_lea... | learni... |
-------------------------------------------------------------------------


: 

In [ ]:
for label in models.keys():
        models[label].fit(X_train_pad, y_train[label])